In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"
import keras
import numpy as np
import io
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

epochs = 100




import numpy as np
from keras.utils import image_dataset_from_directory


# Load datasets using Keras utilities
batch_size = 32
img_size = (1920, 1920)



val_ds = image_dataset_from_directory(
    "valid-versus-invalid/valid",
    label_mode="categorical",
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale",
    pad_to_aspect_ratio=True,
)

In [ ]:
model = keras.models.load_model('models/valid-v-invalid.keras', compile=False)
model.compile(
    loss=keras.losses.CategoricalCrossentropy(), 
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)
print(model.evaluate(val_ds, verbose=1))

In [ ]:
model = keras.models.load_model('models/valid-versus-invalid_best_val_loss.keras', compile=False)
model.compile(
    loss=keras.losses.CategoricalCrossentropy(), 
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)
print(model.evaluate(val_ds, verbose=1))

In [ ]:
val_ds.class_names

In [ ]:
model.layers

In [ ]:
rescaling_layer = model.get_layer(name='rescaling_layer')
image_net_layer = model.get_layer(name='resnet_18_imagenet')
gap_layer = model.get_layer(name='global_average_pooling2d')
dense_layer = model.get_layer(name='dense')

In [ ]:
image_net_layer.summary()

In [ ]:
import keras_hub

In [ ]:
new_backbone = keras_hub.models.ResNetBackbone.from_preset(preset='resnet_18_imagenet', load_weights=False, image_shape=(640, 640, 1))

In [ ]:
new_backbone.summary()

In [ ]:
for old_layer, new_layer in zip(image_net_layer.layers, new_backbone.layers):
    if old_layer.name == new_layer.name and old_layer.trainable and new_layer.trainable:
        new_layer.set_weights(old_layer.get_weights())
        print(old_layer.name)